# Constants

In [60]:
V_CC = 15
K_N = 5e-3
V_TN = -2
LAMBDA = 0.01

R_1 = 1e3
R_2 = 6500
R_SS = 80e3
R_L = 300

In [61]:
def par_res(*res: list[float]) -> float:
    return sum(map(lambda x: x**-1, res))**-1

# DC Analysis

In [62]:
import sympy

I_D = sympy.symbols("I_D")

V_G = V_CC * R_2 / (R_2 + R_1)
print(f"V_G: {V_G}")
V_D = V_CC
print(f"V_D: {V_D}")

V_S = I_D * R_SS
I_D_expr = K_N * (V_G - V_S - V_TN)**2
display(I_D_expr)
I_D1, I_D2 = sympy.solve(I_D_expr - I_D, I_D)
print(f"Potential I_D: {I_D1}, {I_D2}")

V_S1 = I_D1 * R_SS
print(f"V_S1: {V_S1}")
V_S2 = I_D2 * R_SS
print(f"V_S2: {V_S2}")

V_GS1 = V_G - V_S1
V_GS2 = V_G - V_S2
V_DS1 = V_D - V_S1
V_DS2 = V_D - V_S2

I_D1_valid = V_DS1 >= (V_GS1 - V_TN) and V_GS1 > V_TN
I_D2_valid = V_DS2 >= (V_GS2 - V_TN) and V_GS2 > V_TN
print(I_D1_valid, I_D2_valid)

assert I_D1_valid or I_D2_valid, "No Valid I_D Found"
if I_D1_valid:
    I_D = I_D1
else:
    I_D = I_D2
print(f"I_D: {I_D}")

V_G: 13.0
V_D: 15


32000000.0*(0.0001875 - I_D)**2

Potential I_D: 0.000185094959979675, 0.000189936290020325
V_S1: 14.8075967983740
V_S2: 15.1949032016260
True False
I_D: 0.000185094959979675


# AC Analysis

In [63]:
from math import sqrt

g_m = 2 * sqrt(K_N * I_D)
r_0 = 1 / (LAMBDA * I_D)
print(f"g_m: {g_m}")
print(f"r_0: {r_0}")

g_m: 0.0019240320162599947
r_0: 540263.224946702


In [66]:
import sympy

V_gs, V_o = sympy.symbols("V_g V_o")

V_o = sympy.solve(par_res(R_L, R_SS, r_0) * g_m * (V_gs - V_o) - V_o, V_o)[0]
V_i = V_g
gain = V_o / V_i
print(gain)

0.364972602291683


In [65]:
import sympy

K_Ns, V_TNs = sympy.symbols("K_N V_TN")
R_1s, R_2s, R_SSs = sympy.symbols("R_1 R_2 R_SS")
I_Ds, V_G, V_S = sympy.symbols("I_D, V_G, V_S")

V_S = I_Ds * R_SSs
I_D_ex = sympy.collect(sympy.expand(K_Ns * (V_G - V_S - V_TNs)**2 - I_Ds), I_Ds)
display(I_D_ex)
# V_G = V_CC * R_2 / (R_2 + R_1)
I_D_ex = I_D_ex.subs({K_Ns: 5e-3, V_G: V_CC * R_2 / (R_2 + R_1), V_TNs: -2, R_1s: R_1, R_2s: R_2, R_SSs: R_SS})
sympy.solve(I_D_ex, I_Ds)

I_D**2*K_N*R_SS**2 + I_D*(-2*K_N*R_SS*V_G + 2*K_N*R_SS*V_TN - 1) + K_N*V_G**2 - 2*K_N*V_G*V_TN + K_N*V_TN**2

[0.000185094959979675, 0.000189936290020325]